# Import dependencies

In [1]:
# import depenencies
import pandas as pd
import numpy as np
import boto3
import sagemaker
import pickle
import os
import torch
import json

# Instantiate SageMaker session, role and bucket

In [2]:
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

# Upload data to S3


In [3]:
# should be the name of directory you created to save your features data
data_dir = 'data'

# set prefix, a descriptive name for a directory  
prefix = 'ingredients'

# upload all data to S3
input_data=sagemaker_session.upload_data(data_dir, bucket, prefix)

---
# Create an Estimator




In [4]:
from sagemaker.pytorch import PyTorch

output_path = f's3://{bucket}/{prefix}'

env = {
    'SAGEMAKER_REQUIREMENTS': 'requirements.txt', 
}

estimator = PyTorch(entry_point='train.py',
                    source_dir='source',
                    role=role,
                    train_instance_count=1,
                    train_instance_type='ml.p3.8xlarge',
                    sagemaker_session=sagemaker_session,
                    output_path=output_path,
                    framework_version='1.5',
                    py_version='py3',
                    debugger_hook_config=False,
                    hyperparameters={
                        'num_tags': 8,
                        'epochs': 7,
                        'lr': 0.00002
                    })


train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


## Train the estimator

In [5]:
%%time

estimator.fit({'train': input_data})

2021-02-16 21:28:43 Starting - Starting the training job...
2021-02-16 21:29:06 Starting - Launching requested ML instancesProfilerReport-1613510922: InProgress
.........
2021-02-16 21:30:27 Starting - Preparing the instances for training...
2021-02-16 21:31:08 Downloading - Downloading input data......
2021-02-16 21:32:08 Training - Downloading the training image.....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-02-16 21:32:50,534 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-02-16 21:32:50,576 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-02-16 21:32:51,999 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-02-16 21:32:52,394 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python -m pip install -r requirements.txt

2021-02-16 21:33:09 Tra

## Deploy the trained model


In [6]:
%%time

from sagemaker.pytorch import PyTorchModel

# Create a model from the trained estimator data
# And point to the prediction script
model = PyTorchModel(model_data=estimator.model_data,
                     role=role,
                     framework_version='1.5',
                     py_version='py3',
                     entry_point='predict.py',
                     source_dir='source')


# deploy model to create a predictor
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')


----------------!CPU times: user 33.5 s, sys: 3.63 s, total: 37.1 s
Wall time: 8min 39s


In [9]:
predictor.endpoint

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'pytorch-inference-2021-02-16-22-42-14-401'